## **ViT DINO Attention Maps**

**DINO ViT Model choice**

You can choose between the following models:
- `vit_small_patch14_518.dinov2`
- `vit_base_patch14_518.dinov2`
- `vit_large_patch14_518.dinov2`
- `vit_small_patch16_224.dino`
- `vit_small_patch8_224.dino`
- `vit_base_patch16_224.dino`
- `vit_base_patch8_224.dino`

Remember to change the image size in the dataset loader accordingly.

In [5]:
# Install the dev extension to use pytorch
! pip install "mlx-image[dev]"

In [1]:
from mlxim.model import create_model
import mlx.core as mx
import mlx.nn as nn
from mlxim.transform import ImageNetTransform
from mlxim.data import FolderDataset, DataLoader
from mlxim.io import read_rgb
from matplotlib import pyplot as plt
import numpy as np
import cv2
import random
from mlxim.io import read_rgb
import torch
import torch.nn.functional as F
import os

In [6]:
dataset = FolderDataset(
    root_dir="../data/test/cat",
    transform=ImageNetTransform(
        train=False,
        img_size=224,
    ),
    verbose=False,
)

In [4]:
model_name = "vit_small_patch8_224.dino"

model = create_model(model_name=model_name)
model.eval();

In [10]:
# random image
idx = random.choice(range(len(dataset)))
image_path = dataset.images[idx]
x = mx.expand_dims(dataset[idx], axis=0)

In [11]:
# computing the attention masks from all heads
_, attn_mask = model(x, attn_masks=True)
attn = attn_mask[-1]

In [12]:
# setting some parameters
_, w, h, _ = x.shape

patch_size = model.patch_size

w_featmap = w // patch_size
h_featmap = h // patch_size

w = w - w % patch_size
h = h - h % patch_size

### **Plot single attention maps**

In [21]:
# apply some threshold to get only most of the attention masses
# threshold = 0.75
threshold = None

# switching to torch because it's easier to work with
attentions = torch.tensor(np.array(attn))

nh = attentions.shape[1]  # number of head

# we keep only the output patch attention
attentions = attentions[0, :, 0, 1:].reshape(nh, -1)

th_attn = None
if threshold is not None:
    # we keep only a certain percentage of the mass
    val, idx = torch.sort(attentions)
    val /= torch.sum(val, dim=1, keepdim=True)
    cumval = torch.cumsum(val, dim=1)
    th_attn = cumval > (1 - threshold)
    idx2 = torch.argsort(idx)
    for head in range(nh):
        th_attn[head] = th_attn[head][idx2[head]]
    th_attn = th_attn.reshape(nh, w_featmap, h_featmap).float()
    # interpolate
    th_attn = F.interpolate(th_attn.unsqueeze(0), scale_factor=patch_size, mode="nearest")[0].detach().cpu().numpy()

attentions = attentions.reshape(nh, w_featmap, h_featmap)
attentions = F.interpolate(attentions.unsqueeze(0), scale_factor=patch_size, mode="nearest")[0].detach().cpu().numpy()

In [ ]:
if th_attn is not None:
    attentions = th_attn

img = read_rgb(image_path)
img = cv2.resize(img, (w, h))
for i, v in enumerate(attentions):
    # Attention from the output token to the input space.
    # mask = v[0, 1:].reshape(grid_size, grid_size).detach().numpy()
    mask = v[..., np.newaxis]
    # result = (mask / mask.max() * img).astype("uint8")
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16, 16))
    ax1.set_title("Original")
    ax2.set_title("Attention Map_%d Layer" % (i + 1))
    _ = ax1.imshow(img)
    _ = ax2.imshow(mask / mask.max())

# Summing up all the attention maps
ax1.set_title("Original")
ax2.set_title("All attention maps")
_ = ax1.imshow(img)
mask = np.sum(attentions, axis=0)
_ = ax2.imshow(mask / mask.max())